<a href="https://colab.research.google.com/github/rko7/SA-Milestone-Project/blob/main/milestone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install pyodbc

In [23]:
import pyodbc
import requests  # Import requests library for making HTTP requests

# Database connection function

In [24]:
def connect_db():
    server = 'your_server_name'
    database = 'your_database_name'
    username = 'your_username'
    password = 'your_password'
    driver = '{ODBC Driver 18 for SQL Server}'

    connection_string = (
        f'DRIVER={driver};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        f'Encrypt=yes;'  # Secure the connection
        f'TrustServerCertificate=no;'  # Require a valid certificate
        f'Connection Timeout=60;'  # Set a timeout for establishing a connection
    )
    connection = pyodbc.connect(connection_string)
    return connection

# Ensure the Books table exists

In [25]:
def setup_table(connection):
    cursor = connection.cursor()
    create_table_sql = """
    IF OBJECT_ID('dbo.Books', 'U') IS NULL
    BEGIN
        CREATE TABLE dbo.Books (
            BookID INT IDENTITY(1,1) PRIMARY KEY,
            Title NVARCHAR(255) NOT NULL,
            Author NVARCHAR(255) NOT NULL,
            Genre NVARCHAR(100),
            PublicationYear INT
        );
    END;
    """
    cursor.execute(create_table_sql)
    connection.commit()
    print("Books table created (or already exists).")
    cursor.close()

# Fetch books from Google Books API

In [26]:
def fetch_books_from_google_api(query):
    api_url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        books = []
        for item in data['items'][:5]:  # Limit to 5 results for demo purposes
            title = item['volumeInfo'].get('title', 'N/A')
            authors = ', '.join(item['volumeInfo'].get('authors', ['Unknown Author']))
            genre = item['volumeInfo'].get('categories', ['N/A'])[0]
            publication_year = item['volumeInfo'].get('publishedDate', 'N/A')[:4]
            books.append((title, authors, genre, int(publication_year) if publication_year.isdigit() else None))
        return books
    else:
        print("Error fetching data from Google Books API.")
        return []

# Insert fetched books into the database

In [27]:
def insert_fetched_books(connection, books):
    cursor = connection.cursor()
    insert_sql = "INSERT INTO Books (Title, Author, Genre, PublicationYear) VALUES (?, ?, ?, ?);"
    for book in books:
        cursor.execute(insert_sql, book)
    connection.commit()
    print("Fetched books from Google API inserted into Books table.")
    cursor.close()

# Retrieve data from the database

In [28]:
def retrieve_data(connection):
    cursor = connection.cursor()
    select_sql = "SELECT * FROM dbo.Books"
    cursor.execute(select_sql)
    rows = cursor.fetchall()
    print("Retrieved data from the Books table:")
    for row in rows:
        print(f"ID: {row.BookID}, Title: {row.Title}, Author: {row.Author}, Genre: {row.Genre}, Year: {row.PublicationYear}")
    cursor.close()

# Update data from the database

In [ ]:
def update_books(connection, book_id, title=None, author=None, genre=None, year=None):
    """
    Updates book details in the Books table.
    - book_id: The ID of the book to update.
    - title, author, genre, year: Fields to update (optional).
    """
    cursor = connection.cursor()
    update_fields = []
    values = []

    if title:
        update_fields.append("Title = ?")
        values.append(title)
    if author:
        update_fields.append("Author = ?")
        values.append(author)
    if genre:
        update_fields.append("Genre = ?")
        values.append(genre)
    if year:
        update_fields.append("PublicationYear = ?")
        values.append(year)

    if not update_fields:
        print("No fields to update.")
        return

    update_sql = f"UPDATE Books SET {', '.join(update_fields)} WHERE BookID = ?"
    values.append(book_id)

    try:
        cursor.execute(update_sql, values)
        connection.commit()
        print(f"Book with ID {book_id} updated successfully.")
    except Exception as e:
        print(f"Error updating book: {e}")
    finally:
        cursor.close()

# Delete sample data from the database

In [29]:
def delete_sample_data(connection):
    cursor = connection.cursor()
    delete_sql = "DELETE FROM Books WHERE Author = 'Sample Author';"
    cursor.execute(delete_sql)
    connection.commit()
    print("Sample data deleted.")
    cursor.close()

# Disconnect from the database

In [30]:
def disconnect_db(connection):
    connection.close()
    print("Database connection closed.")

if __name__ == "__main__":
    conn = connect_db()
    setup_table(conn)
    books = fetch_books_from_google_api("Python programming")
    insert_fetched_books(conn, books)
    update_books(conn, 1, title="Updated Book Title")
    retrieve_data(conn)
    delete_sample_data(conn)
    disconnect_db(conn)


Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")